In [1]:
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import mean_squared_error
from string import punctuation
import os
from tqdm import tqdm
import re
import numpy as np
import time
import pickle
from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd
from glob import glob
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

In [2]:
from nltk.corpus import wordnet

In [3]:
from nltk.stem.porter import *

# 讀取檔案

In [4]:
train_text=pd.read_csv('train_clear.csv')
test_text=pd.read_csv('test.csv')

In [5]:
train_text

,tweet_id,text,identification,emotion,text_clear
0,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation,people post add snapchat dehydrated cuz man thats
1,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness,trump dangerous freepress world trumplegacy cnn
2,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear,issa stalk tasha 😂😂😂
3,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy,thx best time tonight story heartbreakingly au...
4,0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation,wait supply liscus
...,...,...,...,...,...
1455558,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,train,joy,im happy nowonder happy 👏👏👏👏👏
1455559,0x38959e,In every circumtance I'd like to be thankful t...,train,joy,circumtance id like thankful almighty jesus ch...
1455560,0x2cbca6,there's currently two girls walking around the...,train,joy,there currently girls walk library handing red...
1455561,0x24faed,"Ah, corporate life, where you can date <LH> us...",train,joy,ah corporate life date relative anachronism jo...


In [7]:
Counter(train_text['emotion'])

Counter({'anticipation': 248935,
         'sadness': 193437,
         'fear': 63999,
         'joy': 516017,
         'anger': 39867,
         'trust': 205478,
         'disgust': 139101,
         'surprise': 48729})

In [12]:
tt_train=train_text.groupby('emotion',group_keys=False).apply(lambda x: x.sample(39800))

In [14]:
tt_train

,tweet_id,text,identification,emotion,text_clear
1321050,0x1f5016,My neighbor threw a bunch of cardboard in my t...,train,anger,neighbor throw bunch cardboard trash fill recy...
1082251,0x35fc92,@BBCDorset people must #trawl the #media for s...,train,anger,people trawl medium getajob
595879,0x1fff0d,"🕘 05:00 11 Sep <LH> 🇺🇸USD = 9,29 🇪🇺EUR = 11,15...",train,anger,🕘 sep 🇺🇸usd 🇪🇺eur 🇨🇦cad 🇬🇧gbp 🇨🇳cny 🇷🇺rub
904994,0x3273ab,<LH> Day 2 with no internet... someone tell m...,train,anger,day internet tell uverse 🤷🏾‍♀️🙄🙄🙄
785373,0x2b5acc,Meeting the Sun ☀️ early morning takes away E...,train,anger,meet sun ☀️ early morning take away ego anger ...
...,...,...,...,...,...
874684,0x2fea61,"Amazing Day. Taking care of business, prepping...",train,trust,amazing day take care business prepping incred...
1058352,0x350e2f,@CharlesFaram just wondering if you have 10kg ...,train,trust,wonder kg bullion hoppellets available maybe h...
639113,0x1eef8c,Some real quality inspiring vids from @DannyWo...,train,trust,real quality inspire vids deffo go binge watch...
37648,0x30c24c,New door opened🙏🏼 <LH> <LH>,train,trust,new door opened🙏🏼


In [15]:
test_text=test_text.drop(['predict'],axis=1)

In [16]:
test_text

,tweet_id,text,identification
0,0x28b412,"Confident of your obedience, I write to you, k...",test
1,0x2de201,"""Trust is not the same as faith. A friend is s...",test
2,0x218443,When do you have enough ? When are you satisfi...,test
3,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test
4,0x26289a,"In these tough times, who do YOU turn to as yo...",test
...,...,...,...
411967,0x2913b4,"""For this is the message that ye heard from th...",test
411968,0x2a980e,"""There is a lad here, which hath five barley l...",test
411969,0x316b80,When you buy the last 2 tickets remaining for ...,test
411970,0x29d0cb,I swear all this hard work gone pay off one da...,test


In [8]:
train_text[['text']]

,text
0,"People who post ""add me on #Snapchat"" must be ..."
1,"@brianklaas As we see, Trump is dangerous to #..."
2,Now ISSA is stalking Tasha 😂😂😂 <LH>
3,@RISKshow @TheKevinAllison Thx for the BEST TI...
4,Still waiting on those supplies Liscus. <LH>
...,...
1455558,I'm SO HAPPY!!! #NoWonder the name of this sho...
1455559,In every circumtance I'd like to be thankful t...
1455560,there's currently two girls walking around the...
1455561,"Ah, corporate life, where you can date <LH> us..."


In [17]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [18]:
def get_label(ans_train):
    labels_ = []
    labels=ans_train[['emotion']]
    labels=labels.values.tolist()
    
    for label in labels:
        if label[0]=='joy':
            labels_.append(0)
        elif label[0]=='anticipation':
            labels_.append(1)
        elif label[0]=='trust':
            labels_.append(2)
        elif label[0]=='sadness':
            labels_.append(3)
        elif label[0]=='disgust':
            labels_.append(4)
        elif label[0]=='fear':
            labels_.append(5)
        elif label[0]=='anger':
            labels_.append(6)
        elif label[0]=='surprise':
            labels_.append(7)

    labels=np.array(labels_)
    
    return labels

In [63]:
def get_text(ans_train):
       
    texts=ans_train[['text']]
    texts= texts['text'].apply(sentence_remove).tolist()
    
    datas=[]
    wnl = WordNetLemmatizer()
    for text in tqdm(texts):

        text = "".join([c for c in text.lower() if c not in punctuation]) #去除標點
        
        tmp = [item for item in text.split(" ") if len(item)>=3 and item!='lh']#去除停用詞
        
        
        #詞性還原
        
        tagged_sent = pos_tag(tmp)
        lemmas_sent = []
        
        for tag in tagged_sent:
            wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
            temp_word=wnl.lemmatize(tag[0], pos=wordnet_pos)            
            lemmas_sent.append(temp_word)
        
        #print(lemmas_sent)
        tmp=" ".join(lemmas_sent)
        datas.append(tmp)
    
    return datas

In [99]:
def sentence_remove(tweet):
    
    r = "[_.!+-=——,$%^，。？、~@#￥%……&*《》<>「」“”‘’（）{}【】□‡()/／\\\[\]\"]"
    
    tweet = re.sub(r'[http|https]*://[a-zA-Z0-9.?/&=:]*', ' ', tweet)  # 刪除網址
    tweet = re.sub(r'#\S+', ' ', tweet)                                # 删除 hash tag 
    tweet = re.sub(r'@\S+', ' ', tweet)                                # 删除 @用户 
    tweet = re.sub(r'\\S+', ' ', tweet)                                # 删除空格換行
    tweet = re.sub(r, ' ', tweet)                                      # 删除特殊符号  
    return tweet

#取樣+資料擴增

sample_train=train_text.sample(frac=0.3)

labels=get_label(sample_train)
text=get_text(sample_train)

X_train, X_test, y_train, y_test = train_test_split(text,labels,test_size=0.2,random_state=20)

counter=Counter() #紀錄目前有哪些詞及他們的頻率

for sent in tqdm(X_train) :
    
    tmp = [item for item in sent.split(" ") if item!='lh' and item!='.' and item not in STOP_WORDS]
        #print(tmp)
    counter.update(tmp)
    
print(np.max(list(counter.values())))
print(np.min(list(counter.values())))
print(len(counter))


mean_value = np.mean(list(counter.values()))
min_value = mean_value/4  # mean_value/2
max_value = mean_value*100  # mean_value*8
counter = {k: v for k, v in counter.items() if v > min_value}
vocabulary = sorted(list(counter.keys()),reverse=True)

print('mean : ',mean_value)

tf = TfidfVectorizer(vocabulary=vocabulary)
tfidf_model=tf.fit(X_train)

X_train=tfidf_model.transform(X_train)
X_test=tfidf_model.transform(X_test)

from imblearn.over_sampling import SMOTE
X_train, y_train = SMOTE().fit_resample(X_train, y_train)


In [87]:
X_train.shape[0]

990896

In [93]:
ar,num=np.unique(y_train,return_counts=True)
print(ar,num)

[0 1 2 3 4 5 6 7] [123862 123862 123862 123862 123862 123862 123862 123862]


In [94]:
from imblearn.under_sampling import TomekLinks
X_res, y_res = TomekLinks().fit_resample(X_train, y_train)

In [96]:
print(X_res.shape[0])
ar,num=np.unique(y_res,return_counts=True)
print(ar,num)

987849
[0 1 2 3 4 5 6 7] [123862 123076 123179 123347 123533 123569 123679 123604]


In [100]:
labels=get_label(train_text)

In [101]:
text=get_text(train_text)

100%|██████████████████████████████████████████████████████████████████████| 1455563/1455563 [09:26<00:00, 2568.25it/s]


In [65]:
text[0:10]

['people who post add snapchat must dehydrate cuz man thats',
 'see trump dangerous freepress around the world what trumplegacy cnn',
 'now issa stalk tasha 😂😂😂',
 'thx for the best time tonight what stories heartbreakingly authentic laughoutloud good',
 'still wait those supply liscus',
 'love know gender',
 'more highlight be be show than actual sport who watch triathlon highlight anyway leaguecup',
 'the ssm debate manufacture fantasy use distract the ignorant mass from their mundane life gender diversity',
 'love suffer love when valium do nothing help love when doctor say that theyve do all they can',
 'can someone tell why feed scroll back the same tweet that saw min ago piss']

In [66]:
train_text['text_clear'][0:10]

0    people post add snapchat dehydrated cuz man thats
1      trump dangerous freepress world trumplegacy cnn
2                                 issa stalk tasha 😂😂😂
3    thx best time tonight story heartbreakingly au...
4                                   wait supply liscus
5                                  love know gender 😢😭
6    highlight show actual sport watch triathlon hi...
7    ssm debate manufacture fantasy distract ignora...
8    love suffer 🙃🙃 love valium help 🙃🙃 love doctor...
9              tell feed scroll tweet saw min ago piss
Name: text_clear, dtype: object

In [13]:
text['text_clear'].fillna('.', inplace = True)

C:\Users\oscar\AppData\Local\Temp\ipykernel_7112\4106972080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['text_clear'].fillna('.', inplace = True)


In [14]:
text=text['text_clear'].tolist()

In [62]:
#train_text.to_csv("train_clear.csv",index=False)

# 特徵工程

In [103]:
#文本分為訓練及驗證
X_train, X_test, y_train, y_test = train_test_split(text,labels,test_size=0.2,random_state=20)

In [104]:
X_train[-1]

'god ultimate plan for your life worth wait for'

In [105]:
counter=Counter() #紀錄目前有哪些詞及他們的頻率

In [106]:
for sent in tqdm(X_train) :
    
    tmp = [item for item in sent.split(" ") if item!='lh' and item!='.']
        #print(tmp)
    counter.update(tmp)

100%|████████████████████████████████████████████████████████████████████| 1164450/1164450 [00:02<00:00, 499162.28it/s]


In [107]:
print(np.max(list(counter.values())))
print(np.min(list(counter.values())))
print(len(counter))

444214
1
217806


In [108]:
mean_value = np.mean(list(counter.values()))
min_value = mean_value/2  # mean_value/2
max_value = mean_value*100  # mean_value*8
counter = {k: v for k, v in counter.items() if v > min_value}
vocabulary = sorted(list(counter.keys()),reverse=True)

mean_value

47.13103863070806

In [109]:
len(vocabulary)

12775

In [110]:
tf = TfidfVectorizer(vocabulary=vocabulary)

In [111]:
tfidf_model=tf.fit(X_train)

In [23]:
import joblib

In [183]:
joblib.dump(tfidf_model,"Tfidf_with_xgboost_V2.model")

['Tfidf_with_xgboost_V2.model']

In [112]:
X_train=tfidf_model.transform(X_train)
X_test=tfidf_model.transform(X_test)

# 模型加載

In [35]:
import optuna
import xgboost as xgb

<1x53592 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

#條參數

In [37]:
def objective(trial, X=X_train, y=y_train):
    """
    A function to train a model using different hyperparamerters combinations provided by Optuna.
    """
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4)
    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 10),
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000, 100),
        #'colsample_bytree': trial.suggest_int('colsample_bytree', 0.5, 1),       
        'gamma': trial.suggest_float("gamma", 1e-8, 1.0, log=True),   
        "learning_rate": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        'objective':'softmax',
        #'tree_method': 'gpu_hist'
    }

    reg = xgb.XGBClassifier(**params)
    model_t=reg.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)], eval_metric='merror',
            verbose=False)
    
    return model_t.score(X_valid,y_valid)
    #return mean_squared_error(y_valid, reg.predict(X_valid), squared=False)

In [38]:
%%time
# Creating Optuna object and defining its parameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials = 10)

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2022-11-20 15:06:14,311] A new study created in memory with name: no-name-7b2dafe0-e046-4141-a00a-6d079ff99120
[I 2022-11-20 15:21:14,561] Trial 0 finished with value: 0.5307591566834128 and parameters: {'max_depth': 9, 'n_estimators': 900, 'gamma': 0.00846302430406976, 'colsample_bytree': 0.25115592401115044}. Best is trial 0 with value: 0.5307591566834128.
E:\env\py39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-11-20 15:41:21,972] Trial 1 finished with value: 0.5328373910429817 and parameters: {'max_depth': 6, 'n_estimators': 1800, 'gamma': 0.00010153528190312215, 'colsample_bytree': 0.6487718106076513}. Best is trial 1 with value: 0.5328373910429817.
E:\env\py39\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn,

Number of finished trials: 10
Best trial parameters: {'max_depth': 9, 'n_estimators': 1500, 'gamma': 0.009804179085702403, 'colsample_bytree': 0.2297635947368334}
Best score: 0.5342436343338056
CPU times: total: 2d 21h 20min 2s
Wall time: 3h 30min 36s


In [116]:
#optune 的 參數

params={'max_depth': 7, 
        'n_estimators': 1500, 
        'gamma': 2.7783540662654385e-06, 
        'colsample_bytree': 0.8169210168405894, 
        'objective':'softmax',
      #'tree_method': 'gpu_hist'
       }

In [36]:
#手動 的 參數

params_2={'max_depth': 9, 'n_estimators': 1500, 'gamma': 0.009804179085702403, 'colsample_bytree': 0.2297635947368334}

In [117]:
reg = xgb.XGBClassifier(**params)

In [118]:
model=reg.fit(X_train, y_train,
        eval_set=[(X_test, y_test)], eval_metric='merror',
        verbose=False)

In [85]:
print('score_ver18:%.5f'%model.score(X_test,y_test))

score_sample_ver:0.54733


In [140]:
print('score_ver19:%.5f'%model.score(X_test,y_test))

score_ver19:0.49055


In [184]:
joblib.dump(model,"xgboost_V2.model")

['xgboost_V2.model']

# 測試區

In [130]:
t_texts=get_text(test_text)

100%|████████████████████████████████████████████████████████████████████████| 411972/411972 [03:07<00:00, 2192.95it/s]


In [121]:
len(t_texts)

411972

In [131]:
t_texts

['confident your obedience write you know that you will even more than ask philemon',
 'trust not the same faith friend someone you trust put faith anyone mistake christopher hitchens',
 'when you have enough when be you satisfied you goal really all about money',
 'god wake you now chase the day',
 'these tough time who you turn your symbol hope',
 'turn out you can recognise people their undies',
 'like how hayvens mommy daddy and the keyboard warrior have jump into everything she can handle anything herself',
 'just love when every single one song just delete themselves this the time this have happen',
 'when can expect season think that would',
 'tbh regret hurt more than step lego',
 'nothing like the delicious taste coffee and french fry',
 'look everything like will go tomorrow',
 'have plenty time hand all work donald trump',
 'be say this for year you be',
 'forgot for the class presentation list but bff already put name down 😭✊🏼👯 mari gots',
 'gonna house sit during the new m

In [81]:
test_text['text_clear']=t_texts

In [95]:
#test_text.to_csv("test_clear.csv",index=False)

In [132]:
tf_data=tf.transform(t_texts)

In [133]:
y_predict=model.predict(tf_data)

In [134]:
print(len(y_predict))

411972


In [135]:
list(y_predict)

[1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 3,
 0,
 0,
 0,
 0,
 4,
 1,
 4,
 1,
 0,
 0,
 0,
 0,
 0,
 4,
 6,
 2,
 0,
 0,
 3,
 3,
 0,
 0,
 3,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 1,
 3,
 0,
 3,
 2,
 3,
 1,
 0,
 0,
 1,
 3,
 0,
 1,
 0,
 1,
 5,
 0,
 0,
 0,
 0,
 1,
 0,
 3,
 0,
 3,
 3,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 4,
 0,
 0,
 3,
 0,
 0,
 4,
 0,
 4,
 2,
 0,
 1,
 0,
 4,
 0,
 0,
 1,
 0,
 0,
 1,
 5,
 3,
 0,
 0,
 3,
 3,
 2,
 3,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 2,
 3,
 2,
 1,
 1,
 0,
 0,
 1,
 1,
 4,
 0,
 2,
 4,
 0,
 3,
 4,
 1,
 2,
 2,
 5,
 0,
 4,
 0,
 0,
 2,
 1,
 4,
 3,
 0,
 0,
 2,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 4,
 2,
 3,
 0,
 0,
 0,
 3,
 3,
 3,
 1,
 3,
 7,
 3,
 0,
 0,
 0,
 1,
 0,
 4,
 2,
 3,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 2,
 5,
 3,
 4,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 4,
 0,
 0,
 5,
 3,
 2,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 3,
 0,
 3,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 3,
 1,


In [47]:
ar,num=np.unique(y_predict,return_counts=True)
print(ar,num)

[0 1 2 3 4 5 6 7] [65677 51045 58611 50815 61491 32111 51587 40635]


In [136]:
rst=test_text

In [137]:
rst['predict']=0

In [138]:
for i in range(0,411972):
    if y_predict[i]==0:
        temp='joy'
        
    elif y_predict[i]==1:
        temp='anticipation'
        
    elif y_predict[i]==2:
        temp='trust'
        
    elif y_predict[i]==3:
        temp='sadness'
        
    elif y_predict[i]==4:
        temp='disgust'
        
    elif y_predict[i]==5:
        temp='fear'
        
    elif y_predict[i]==6:
        temp='anger'
        
    elif y_predict[i]==7:
        temp='surprise'
        
    rst.at[i,'predict']=temp

In [95]:
rst

,tweet_id,text,identification,predict
0,0x28b412,"Confident of your obedience, I write to you, k...",test,anticipation
1,0x2de201,"""Trust is not the same as faith. A friend is s...",test,trust
2,0x218443,When do you have enough ? When are you satisfi...,test,joy
3,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test,anticipation
4,0x26289a,"In these tough times, who do YOU turn to as yo...",test,trust
...,...,...,...,...
411967,0x2913b4,"""For this is the message that ye heard from th...",test,joy
411968,0x2a980e,"""There is a lad here, which hath five barley l...",test,anticipation
411969,0x316b80,When you buy the last 2 tickets remaining for ...,test,anticipation
411970,0x29d0cb,I swear all this hard work gone pay off one da...,test,joy


In [77]:
rst

,tweet_id,text,identification,predict
0,0x28b412,"Confident of your obedience, I write to you, k...",test,anticipation
1,0x2de201,"""Trust is not the same as faith. A friend is s...",test,anticipation
2,0x218443,When do you have enough ? When are you satisfi...,test,joy
3,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test,joy
4,0x26289a,"In these tough times, who do YOU turn to as yo...",test,trust
...,...,...,...,...
411967,0x2913b4,"""For this is the message that ye heard from th...",test,joy
411968,0x2a980e,"""There is a lad here, which hath five barley l...",test,anticipation
411969,0x316b80,When you buy the last 2 tickets remaining for ...,test,anticipation
411970,0x29d0cb,I swear all this hard work gone pay off one da...,test,joy


In [139]:
to_sub=rst.drop(['text','identification'],axis=1)
to_sub.columns = ['id','emotion']
to_sub.to_csv('submission_20.csv',index=0)

In [53]:
to_sub

,id,emotion
0,0x28b412,trust
1,0x2de201,trust
2,0x218443,anticipation
3,0x2939d5,anticipation
4,0x26289a,trust
...,...,...
411967,0x2913b4,anticipation
411968,0x2a980e,anticipation
411969,0x316b80,sadness
411970,0x29d0cb,anger
